In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import pandas as pd

In [2]:
#Grab the schedule links
ScheduleLink=[]
for year in range(1990,2019):
    #Get the Link and request HTML
    Link = 'https://www.baseball-reference.com/leagues/MLB/'+str(year)+'-schedule.shtml'
    soup = BeautifulSoup(requests.get(Link).text, "html.parser")
    Sch = soup.find_all("div", "section_wrapper")
    
    #Get the Season links.
    SeasonLinks=Sch[0].find_all('a')
    BoxscoreLinksSeason = [x.get('href') for x in SeasonLinks if x.get_text()=='Boxscore']
    
    #Append the links.
    if year < 2018:
        PostSeasonLinks=Sch[1].find_all('a')
        BoxscoreLinksPostSeason = [x.get('href') for x in PostSeasonLinks if x.get_text()=='Boxscore']
        ScheduleLink = ScheduleLink+BoxscoreLinksSeason+BoxscoreLinksPostSeason
    else:
        ScheduleLink = ScheduleLink+BoxscoreLinksSeason

In [3]:
#Get the schedule page
WeatherData = []
for link in ScheduleLink:
    Link = 'https://www.baseball-reference.com'+link
    soup = BeautifulSoup(requests.get(Link).text, "html.parser")

    #Date HomeTeam AwayTeam GameTime Temperature

    #Date
    shortDate=ScheduleLink[0].split('/')[3].split('.')[0][3:]
    Date = shortDate[:4]+'-'+shortDate[4:6]+'-'+shortDate[6:8]

    #Clean stupid links
    stupidLinks = soup.find_all("div", "linescore_wrap")[0].find_all('tbody')[0].find_all('a')
    Links = [x for x in stupidLinks if "teams" in x.get('href')]

    #Teams
    AwayTeam = Links[0].get_text()
    HomeTeam = Links[1].get_text()

    #Weather
    try:
        strings = str(soup.find_all('body', 'br')[0].find_all('div','section_wrapper',id='all_6309354931')[0])
        Weather=strings.split('<div>')[-1].split("</strong>")[1].split('<')[0].strip()

        if Weather == 'In Dome.':
            Temp = 73.4
        else:
            try:
                Temp = float(Weather.split('&')[0])
            except:
                Temp = 'NA'
    except:
        Temp = 'NA'

    Row = [Date, HomeTeam, AwayTeam, Temp]
    WeatherData.append(Row)

In [4]:
WeatherData = pd.DataFrame(data=WeatherData, columns=['Date', 'Home Team', 'Away Team', 'Temp'])

In [5]:
WeatherData.to_csv('./Schedules/TempData/HistoricalWeather.csv')